# **CamemBERT**

[CamemBERT](https://camembert-model.fr/) est un modèle de traitement automatique de la langue Française, basé sur l'architecture [RoBERTa](https://ai.facebook.com/blog/roberta-an-optimized-method-for-pretraining-self-supervised-nlp-systems/) développée par Facebook AI en 2019, dédiée à l'Anglais. RoBERTa est lui même basé sur [BERT](https://fr.wikipedia.org/wiki/BERT_(mod%C3%A8le_de_langage)) qui a été développé par Google en 2018.  
CamemBERT est donc un cousin Français de BERT, qui a pu voir le jour lorsque les équipes de Facebook associés aux chercheurs de [l'INRIA](https://www.inria.fr/fr) ont rendu public ce modèle pré-entrainé sur 138GB de texte Français.  
CamemBERT a été pré-entraîné sur un corpus francophone et avec des hyper-paramètres différents découverts et testés pour la première fois par l’équipe de Facebook. Le choix de ces hyper-paramètres était tellement réussi que l’entreprise a annoncé le sortie d’un “nouveau” modèle baptisé RoBERTa. Pourtant, il n’y a rien de nouveau dans RoBERTA qui comme CamemBERT reste une copie de BERT. Voici ces hyper-paramètres:  
*  CamemBERT choisit les mots à prédire de manière dynamique, c’est-à-dire, non pas lors du pré-processing des données en entrée, mais lors de forward pass, en masquant au hasard certains mots d’une séquence.  
*  Il utilise un batch size différent: ~8 000 contre 256 dans le cas de BERT.  
*  CamemBERT a un seul objectif de pré-entrainement: prédiction des “mots masqués” d’une séquence. BERT en avait deux : prédiction des “mots masqués” et de la phrase suivante d’une séquence. Ce dernier objectif s’est avéré improductif pour l’entrainement.

# **Exemple d'utlisation de CamemBERT**

CamemBERT a été entrainé dans le but de prédire des "mots masqués" dans un texte. Nous allons voir un exemple de ce que peut faire ce modèle pré-entrainé.

Commençons par installer sur la machine les modules python dont nous aurons besoin et qui ne sont pas pré-installés. Ensuite nous importons les bibliothèques.  
En particulier, nous utiliserons la bibliothèque [Transformers](https://huggingface.co/transformers/#) créée par [Hugging Face](https://huggingface.co/). Cette bibliothèque contient des centaines de modèles pré-entrainés pour réaliser des opérations sur les données textuelles, comme la classification, l'extraction d'informations, le "questions-réponses", la traduction, ...

In [34]:
!pip3 install transformers --quiet
!pip3 install transformers sentencepiece

In [35]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

from transformers import TFCamembertForMaskedLM
from transformers import AutoTokenizer

Définisson la variable tokenizer qui permettra d'instancier le tokenizer pour CamemBERT fourni par la librairie "transformers" :

In [36]:
tokenizer = AutoTokenizer.from_pretrained("jplu/tf-camembert-base",use_fast=False)

ValueError: ignored

Définissons maintenant la phrase que nopus souhaitons compléter puis utlisons le tokenizer de la bibliothèque "transformers" pour préparer le texte :

In [ ]:
phrase = "L'intelligence artificielle va mener à la <mask> du monde !"
output_tokenizer = tokenizer.encode_plus(phrase, max_length=100, padding="longest", truncation=True, return_tensors='tf')
output_tokenizer

Récupérons la position du masque (mask) dans la séquence retournée par le tokenizer :

In [ ]:
mask_index = (output_tokenizer['input_ids'][0].numpy() == tokenizer.mask_token_id).nonzero()
mask_index = np.reshape(mask_index,(1))[0]
mask_index

Instancions maintenant le modèle CamemBERT pré-entrainé avec le jeu de données de base pour Tensorflow :

In [ ]:
model = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")

Puis lançons le modèle sur la phrase :

In [ ]:
output = model(output_tokenizer['input_ids'])[0]
output

On récupère les (32005 !) valeurs en sortie du modèle correspondants à l'emplacement du masque :

In [ ]:
output  = output[0, mask_index, :]
output

Puis on applique une fonctioon d'activation Soft-Max afin de normaliser les probabilités sur chaque valeurs :

In [ ]:
proba = tf.nn.softmax(output)
proba

On récupère ensuite les valeurs et les indices des 5 plus grandes probabilités parmi ces 32005 probabilités :

In [ ]:
top_proba, top_indices = tf.math.top_k(proba,k=5)
print(top_proba)
print(top_indices)

On va chercher ensuite les valeurs numériques des mots correspondants aux emplacements de ces probibilités les plus fortes, puis on reconvertit ces valeurs en mots réels (fonction inverse du tokenizer) :

In [ ]:
topk_predicted_token_bpe2 = " ".join([tokenizer.convert_ids_to_tokens(int(tf.keras.backend.get_value(top_indices[i]))) for i in range(len(top_indices))])
topk_predicted_token_bpe2 

On place dans la variable "mask" le mot clé utlisé dans la phrase pour le masque :

In [ ]:
mask = tokenizer.mask_token
mask

On sépare les résultats obtenus :

In [ ]:
topk_predicted_token_bpe2.split(" ")

Puis on remplace le masque dans la phrase initiale, en utlisant toutes les possibilités trouvées :

In [ ]:
topk_filled_outputs_ = []
for index2, predicted_token_bpe2 in enumerate(topk_predicted_token_bpe2.split(" ")):
  predicted_token_ = predicted_token_bpe2.replace("\u2581", " ")
  if " {0}".format(tokenizer.mask_token) in phrase:
    topk_filled_outputs_.append((phrase.replace(" {0}".format(tokenizer.mask_token), predicted_token_)))
  else:
    topk_filled_outputs_.append((phrase.replace(tokenizer.mask_token, predicted_token_)))

topk_filled_outputs_